# TPS July First Model(LightGBM)の作成

[TPS July](https://www.kaggle.com/c/tabular-playground-series-jul-2021) に対してデータチューニングを一部行い、XGBoostで推論を行うモデルを作成する.

今回作成するモデルとしては以下の方針で作成する.

- 説明変数
    - 時刻データ...**パラメータチューニングで追加**
    - 温度データ(絶対値)
    - 湿度データ(絶対値)
    - sensor1
    - sensor2
    - sensor3
    - sensor4
    - sensor5

- ターゲットデータ
    - 一酸化炭素
    - ベンゼン
    - 窒素酸化物
    
また、[このディスカッション](https://www.kaggle.com/anjalianupam/tps-july-eda)から、相関関係が強いもの(絶対値0.5以上)を推論で使用するように変更する.

そして、
- benzene⇒sensor_2と相関が強い
- carbon⇒benzeneと相関が強い
- nitrogen⇒carbonと相関が強い

ということから、それぞれのターゲットも推論に使用する.

## 1. インポート+データセット準備

**インポート**

In [ ]:
# 基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 機械学習ライブラリ
from sklearn.model_selection import train_test_split

# light GBM の準備
! pip install lightgbm
import lightgbm as lgb

# Warningがうるさいのできる
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

**データセット読み込み**

In [ ]:
# データセットの読み込み
train_df = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')

## 2. データチューニング 

### 2.1 全体の前処理

二つのデータセットをまとめて前処理をするので、学習用と予測用を判断するラベルを付けておく。

In [ ]:
# データセットラベルを付与
train_df['Dataset'] = 'train'
test_df['Dataset'] = 'test'

# 一つにまとめる
all_df = pd.concat([train_df, test_df])

特徴変数とターゲットをここで定義しておく

In [ ]:
# 特徴変数とターゲット変数の定義
feature_cols_origin = [
    'deg_C',
    'absolute_humidity',
    'relative_humidity',
    'sensor_1',
    'sensor_2',
    'sensor_3',
    'sensor_4',
    'sensor_5',
]
feature_cols_benzene = [
    'absolute_humidity',
    'deg_C',
    'sensor_1',
    'sensor_2',
    'sensor_3',
    'sensor_4',
    'sensor_5',
    'oclock',
    'is_weekend',
    'before_deg_C_1',
    'before_deg_C_2',
    'before_deg_C_3',
    'before_deg_C_4',
    'before_deg_C_5',
    'before_abs_hum_1',
    'before_abs_hum_2',
    'before_abs_hum_3',
    'before_abs_hum_4',
    'before_abs_hum_5',
]
feature_cols_carbon = [
    'absolute_humidity',
    'deg_C',
    'sensor_1',
    'sensor_2',
    'sensor_3',
    'sensor_4',
    'sensor_5',
    'oclock',
    'is_weekend',
    'before_deg_C_1',
    'before_deg_C_2',
    'before_deg_C_3',
    'before_deg_C_4',
    'before_deg_C_5',
    'before_abs_hum_1',
    'before_abs_hum_2',
    'before_abs_hum_3',
    'before_abs_hum_4',
    'before_abs_hum_5',
#     'target_benzene',
]
feature_cols_nitrogen = [
    'absolute_humidity',
    'deg_C',
    'sensor_1',
    'sensor_2',
    'sensor_3',
    'sensor_4',
    'sensor_5',
    'oclock',
    'is_weekend',
    'before_deg_C_1',
    'before_deg_C_2',
    'before_deg_C_3',
    'before_deg_C_4',
    'before_deg_C_5',
    'before_abs_hum_1',
    'before_abs_hum_2',
    'before_abs_hum_3',
    'before_abs_hum_4',
    'before_abs_hum_5',
#     'target_carbon_monoxide'
]

target_cols = [
    'target_carbon_monoxide',
    'target_benzene',
    'target_nitrogen_oxides'
]

### 2.2 時刻データを付加

時刻に依存関係がある事がわかっているので、`date_time`の時刻、上2ケタのデータを時刻データを抽出し、`oclock`とする。

In [ ]:
all_df['oclock'] = all_df['date_time'].str[-8:-6].astype('int16')
all_df['oclock']

### 2.3. 週末データの追加

週末かどうかによって大気汚染の影響が変わって来ると考えられるため、このデータを追加

In [ ]:
all_df["is_weekend"] = (pd.to_datetime(all_df["date_time"]).dt.dayofweek>=5).astype("int")

### 2.4. 1~5時間前の温度データ、湿度データをラグデータとして取り出す

風が強くなったり、雨が降ったりすると汚染が和らぐとの結果があるとのこと。

風、雨=＞気圧変化が発生=＞直近の温度変化

という関係性が考えられるので、5時間前までの温度差を特徴変数として取り出す。
（ここで最初の5時間分のデータはNanが発生するので、ひとまず除く）

In [ ]:
all_df['before_deg_C_1'] = all_df['deg_C'].shift(1)
all_df['before_deg_C_2'] = all_df['deg_C'].shift(2)
all_df['before_deg_C_3'] = all_df['deg_C'].shift(3)
all_df['before_deg_C_4'] = all_df['deg_C'].shift(4)
all_df['before_deg_C_5'] = all_df['deg_C'].shift(5)
all_df['before_deg_C_5'] = all_df['deg_C'].shift(5)
all_df['before_abs_hum_1'] = all_df['absolute_humidity'].shift(1)
all_df['before_abs_hum_2'] = all_df['absolute_humidity'].shift(2)
all_df['before_abs_hum_3'] = all_df['absolute_humidity'].shift(3)
all_df['before_abs_hum_4'] = all_df['absolute_humidity'].shift(4)
all_df['before_abs_hum_5'] = all_df['absolute_humidity'].shift(5)
all_df['before_abs_hum_5'] = all_df['absolute_humidity'].shift(5)
all_df = all_df.iloc[5:]
all_df

### 季節ラベルを追加

### targetデータをほかのtargetデータの推論データとして使用する

### 2.6. データの正規化

minmaxnormを用いて説明変数全て正規化を行う

In [ ]:
import numpy as np

def minmax_norm(x, axis=None):
    min = x.min(axis=axis, keepdims=True)
    max = x.max(axis=axis, keepdims=True)
    result = (x-min)/(max-min)
    return result

for feature in feature_cols_origin:
    all_df[feature] = minmax_norm(all_df[feature].to_numpy())
all_df

## 3. モデルの作成

Light GBMを用いた学習を実施する.

- 参考アドレス
https://htomblog.com/python-lightgbm#toc2

### 3.1 モデルの作成

In [ ]:
# 訓練デvalidータと評価データを準備する
all_train = all_df[all_df['Dataset']=='train']
all_train = all_train.drop(columns=['Dataset'])
train, valid = train_test_split(all_train, random_state=42)
X_train_carbon, y_train_carbon = train[feature_cols_carbon], train[target_cols[0]]
X_train_benzene, y_train_benzene = train[feature_cols_benzene], train[target_cols[1]]
X_train_nitrogen, y_train_nitrogen = train[feature_cols_nitrogen], train[target_cols[2]]
X_valid_carbon, y_valid_carbon = valid[feature_cols_carbon], valid[target_cols[0]]
X_valid_benzene, y_valid_benzene = valid[feature_cols_benzene], valid[target_cols[1]]
X_valid_nitrogen, y_valid_nitrogen = valid[feature_cols_nitrogen], valid[target_cols[2]]

# X_train, y_train_carbon, y_train_benzene, y_train_nitrogen  = train[feature_cols], train[target_cols[0]], train[target_cols[1]], train[target_cols[2]]
# X_valid, y_valid_carbon, y_valid_benzene, y_valid_nitrogen  = valid[feature_cols], valid[target_cols[0]], valid[target_cols[1]], valid[target_cols[2]]

# LightGBMにはこのコンペティションで採用されてる評価関数rmsleが実装されていないため、
# y ⇒ t (=np.log1p(y))
# を適用したtを用いて学習を実施させる
# 参考...https://www.guruguru.science/competitions/13/discussions/cbb736e9-f0f7-4847-811e-fe038e8ed0e8/
t_train_carbon, t_train_benzene, t_train_nitrogen = np.log1p(y_train_carbon), np.log1p(y_train_benzene), np.log1p(y_train_nitrogen)
t_valid_carbon, t_valid_benzene, t_valid_nitrogen = np.log1p(y_valid_carbon), np.log1p(y_valid_benzene), np.log1p(y_valid_nitrogen)

# Light GBM用データセットを準備
train_carbon = lgb.Dataset(X_train_carbon, t_train_carbon)
valid_carbon = lgb.Dataset(X_valid_carbon, t_valid_carbon)
train_benzene = lgb.Dataset(X_train_benzene, t_train_benzene)
valid_benzene = lgb.Dataset(X_valid_benzene, t_valid_benzene)
train_nitrogen = lgb.Dataset(X_train_nitrogen, t_train_nitrogen)
valid_nitrogen = lgb.Dataset(X_valid_nitrogen, t_valid_nitrogen)

# モデルの学習条件の定義
params = {
    "objective": "regression",
    "metric": "rmse"
}

# 学習の実施
model_carbon = lgb.train(
    params = params,
    train_set = train_carbon,
    valid_sets = [train_carbon, valid_carbon],
    num_boost_round = 300,
    verbose_eval=20
)
model_benzene = lgb.train(
    params = params,
    train_set = train_benzene,
    valid_sets = [train_benzene, valid_benzene],
    num_boost_round = 300,
    verbose_eval=20
)
model_nitrogen = lgb.train(
    params = params,
    train_set = train_nitrogen,
    valid_sets = [train_nitrogen, valid_nitrogen],
    num_boost_round = 300,
    verbose_eval=20
)

## 4. 作成したモデルでtestデータを推論 + 提出

### 4.1. 推論の実施

In [ ]:
# テストデータの準備
all_test = all_df[all_df['Dataset']=='test']
all_test = all_test.drop(columns=['Dataset'])
X_test_carbon = all_test[feature_cols_carbon]
X_test_benzene = all_test[feature_cols_benzene]
X_test_nitrogen = all_test[feature_cols_nitrogen]

# 推論の実施
t_predict_carbon = model_carbon.predict(X_test_carbon)
t_predict_benzene = model_benzene.predict(X_test_benzene)
t_predict_nitrogen = model_nitrogen.predict(X_test_nitrogen)

# 推論結果はtを推論しているため、
# t ⇒ y (=np.expm1(y))
# でyを計算する
predict_carbon = np.expm1(t_predict_carbon)
predict_benzene = np.expm1(t_predict_benzene)
predict_nitrogen = np.expm1(t_predict_nitrogen)

# submissionデータの作成
date_time = all_test['date_time']
submission_data = pd.DataFrame(np.array([date_time, predict_carbon, predict_benzene, predict_nitrogen]).T,
                              columns=['date_time', 'target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'])
submission_data

### 4.2. 推論結果を簡単に描画して確認

In [ ]:
for col in feature_cols_origin:
    fig = plt.figure(figsize=[24, 10])

    x = pd.to_datetime(train_df['date_time'], format='%Y-%m-%d %H:%M:%S').to_list()
    y = train_df[col].to_list()
    plt.plot(x, y, label='train')

    x = pd.to_datetime(test_df['date_time'], format='%Y-%m-%d %H:%M:%S').to_list()
    y = test_df[col].to_list()
    plt.plot(x, y, label='test')

    plt.xlim([pd.to_datetime(train_df.iloc[0]['date_time'], format='%Y-%m-%d %H:%M:%S')
          , pd.to_datetime(test_df.iloc[-1]['date_time'], format='%Y-%m-%d %H:%M:%S')])
    plt.xlabel('date time')
    plt.ylabel(col)
    plt.legend()
    plt.show()

for col in target_cols:
    fig = plt.figure(figsize=[24, 10])

    x = pd.to_datetime(train_df['date_time'], format='%Y-%m-%d %H:%M:%S').to_list()
    y = train_df[col].to_list()
    plt.plot(x, y, label='train')

    x = pd.to_datetime(submission_data['date_time'], format='%Y-%m-%d %H:%M:%S').to_list()
    y = submission_data[col].to_list()
    plt.plot(x, y, label='submission_data')

    
    plt.xlim([pd.to_datetime(train_df.iloc[0]['date_time'], format='%Y-%m-%d %H:%M:%S')
              , pd.to_datetime(test_df.iloc[-1]['date_time'], format='%Y-%m-%d %H:%M:%S')])
    plt.xlabel('date time')
    plt.ylabel(col)
    plt.legend()
    plt.show()

### 4.4. 提出データをCSVに出力

In [ ]:
submission_data.to_csv('submission_first_model.csv', index=False)

以上